In [22]:
import sys
from functions import *
import cmapy

os.environ["CUDA_VISIBLE_DEVICES"] = "3"  

In [23]:
savetest = True
if savetest == True:
    testorval = "test"
    seis_path = 'seistest.npy'
else:
    testorval = "val"
    seis_path = 'seismicval.npy'

In [24]:
seis = np.load(seis_path)

In [25]:
IL, Z, XL = seis.shape
print(IL, Z, XL)

703 1537 3174


In [28]:
im_height = Z
im_width = XL
splitsize = 96
stepsize = 48 #overlap half
overlapsize = splitsize-stepsize

In [29]:
horizontal_splits_number = int(np.ceil((im_width)/stepsize))
print(horizontal_splits_number)
width_after_pad = stepsize*horizontal_splits_number+2*overlapsize
print(width_after_pad)
left_pad = int((width_after_pad-im_width)/2)
right_pad = width_after_pad-im_width-left_pad
print(left_pad,right_pad)

vertical_splits_number = int(np.ceil((im_height)/stepsize))
print(vertical_splits_number)
height_after_pad = stepsize*vertical_splits_number+2*overlapsize
print(height_after_pad)
top_pad = int((height_after_pad-im_height)/2)
bottom_pad = height_after_pad-im_height-top_pad
print(top_pad,bottom_pad)

67
3312
69 69
33
1680
71 72


In [30]:
horizontal_splits_number = horizontal_splits_number+1
print(horizontal_splits_number)
vertical_splits_number = vertical_splits_number+1
print(vertical_splits_number)

68
34


In [31]:
halfoverlapsize = int(overlapsize/2)
print(halfoverlapsize)

24


In [32]:
print(len(seis))

703


In [33]:
t_start = time.time()
X_list = []
for i in range(0,len(seis),5):
    img = seis[i]
    X_list.extend(split_Image(img,True,top_pad,bottom_pad,left_pad,right_pad,splitsize,stepsize,vertical_splits_number,horizontal_splits_number))
#     break
print(len(X_list))
print(X_list[0].shape)
print("read images in {} sec".format(time.time()-t_start))
X = np.asarray(X_list)
print(X.shape)
print("read images in {} sec".format(time.time()-t_start))

325992
(96, 96)
read images in 20.060105800628662 sec
(325992, 96, 96)
read images in 36.8176155090332 sec


In [ ]:
X = X.astype(np.float32)
X = np.expand_dims(X,1)
print(X.shape)

In [ ]:
# idea from: https://www.kaggle.com/erikistre/pytorch-basic-u-net
class faultsDataset(torch.utils.data.Dataset):

    def __init__(self,preprocessed_images):
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
        self.images = preprocessed_images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        return image


In [ ]:

faults_dataset_test = faultsDataset(X)

batch_size = 64

test_loader = torch.utils.data.DataLoader(dataset=faults_dataset_test, 
                                           batch_size=batch_size, 
                                           shuffle=False)

In [ ]:
best_iou_threshold = 0.5
model_name = "unet" # unet deeplab rcf
currentpath = "/sgai_overlap/"
mergemethod = "smooth" # smooth average crop
device = torch.device("cuda")
if model_name == "unet":
    from model_zoo.UNET import Unet
    model = Unet()
    print("use model Unet")
    modelname = "unet_LRoverlap_0.5Loss12_0.5Loss3_addafterepoch4_seed1"
    best_model_fpath = '{}/models/{}.model'.format(currentpath,modelname) 
    save_path = '{}/overlap_clean_predictions/{}_{}emerge_{}'.format(currentpath,modelname,mergemethod,testorval)
elif model_name == "deeplab":
    from model_zoo.DEEPLAB.deeplab import DeepLab
    model = DeepLab(backbone='mobilenet', num_classes=1, output_stride=16)
    print("use model DeepLab")
    modelname = "deeplab_withoutLRoverlap_paired_seed236"
    best_model_fpath = '{}/models/{}.model'.format(currentpath,modelname) 
    save_path = '{}/overlap_clean_predictions/{}_{}emerge_{}'.format(currentpath,modelname,mergemethod,testorval)
elif model_name == "rcf":
    from model_zoo.RCF import RCF
    model = RCF()
    print("use model RCF")
    modelname = "rcf_LRoverlap_0.5Loss12_0.5Loss3_addafterepoch10_seed236"
    best_model_fpath = '{}/models/{}.model'.format(currentpath,modelname) 
    save_path = '{}/overlap_clean_predictions/{}_{}emerge_{}'.format(currentpath,modelname,mergemethod,testorval)
print(best_model_fpath)
print(save_path)
model.load_state_dict(torch.load(best_model_fpath, map_location="cuda:0"))  # Choose whatever GPU device number you want
model.to(device)
summary(model, (1, splitsize, splitsize))

In [ ]:
UPPER_BOUND = 800
LOWER_BOUND = 1300
print("UPPER_BOUND",UPPER_BOUND)
print("LOWER_BOUND",LOWER_BOUND)
    
def saveResults(save_path, test_loader):     
    os.makedirs(save_path, exist_ok=True)

    test_predictions = []
    imageNo = -1
    for images in test_loader:
        images = Variable(images.cuda())
        outputs = model(images)
        y_preds = outputs
        if model_name == "rcf":
            y_preds = outputs[-1]
        test_predictions.extend(y_preds.detach().cpu())
        if len(test_predictions)>=vertical_splits_number*horizontal_splits_number:
            imageNo = imageNo +1
            tosave = torch.stack(test_predictions).detach().cpu().numpy()[0:vertical_splits_number*horizontal_splits_number]
            test_predictions = test_predictions[vertical_splits_number*horizontal_splits_number:]

            if mergemethod == "smooth":
                WINDOW_SPLINE_2D = window_2D(window_size=splitsize, power=2)
                tosave = np.moveaxis(tosave,-3,-1)
                tosave = np.array([patch * WINDOW_SPLINE_2D for patch in tosave])
                tosave = tosave.reshape((vertical_splits_number, horizontal_splits_number, splitsize,splitsize,1))
                recover_Y_test_pred = recover_Image2(tosave, (im_height,im_width,1), left_pad,right_pad,top_pad,bottom_pad,overlapsize)
                os.makedirs(save_path, exist_ok=True)
                np.save(os.path.join(save_path,"{}".format(imageNo)),np.squeeze(recover_Y_test_pred)[UPPER_BOUND:LOWER_BOUND,:])   
            elif mergemethod == "average":
                tosave = tosave.reshape((vertical_splits_number, horizontal_splits_number, splitsize,splitsize,1))
                recover_Y_test_pred = recover_Image(tosave, (im_height,im_width,1), left_pad,right_pad,top_pad,bottom_pad, overlapsize)
                os.makedirs(save_path, exist_ok=True)
                np.save(os.path.join(save_path,"{}".format(imageNo)),np.squeeze(recover_Y_test_pred)[UPPER_BOUND:LOWER_BOUND,:])
            elif mergemethod == "crop":
                tosave= crop2(tosave,48,48)
                tosave = tosave.reshape((vertical_splits_number, horizontal_splits_number, 48,48,1))
                recover_Y_test_pred = recover_Image(tosave, (im_height,im_width,1), left_pad-halfoverlapsize,right_pad-halfoverlapsize,top_pad-halfoverlapsize,bottom_pad-halfoverlapsize,0)
                os.makedirs(save_path, exist_ok=True)
                np.save(os.path.join(save_path,"{}".format(imageNo)),np.squeeze(recover_Y_test_pred)[UPPER_BOUND:LOWER_BOUND,:])
            else:
                print("please enter an valida merge method")
                        

print("save")
t1 = time.time()
saveResults(save_path, test_loader)
t2 = time.time()
print('save in {} sec'.format(t2-t1))

In [ ]:

import numpy as np
import os

In [ ]:
a = np.load(os.path.join(save_path,"20.npy"))

In [ ]:
import cv2
import cmapy

In [ ]:
heatmap_img = cv2.applyColorMap((a*255).astype(np.uint8), cmapy.cmap('jet_r'))
plt.figure(figsize=(12,2))
plt.imshow(heatmap_img[:,:,:])
plt.axis('off')
plt.savefig("{}/overlap_predictions_example/{}_XL100_{}merge_{}.png".format(currentpath,modelname,mergemethod,testorval),bbox_inches="tight")
plt.show()